In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-9d62  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import csv
# import gzip

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 11 08:02 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'porjectirofererez111'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix='multi')
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!***

# Building 3 inverted indexes

Here, we read the entire corpus to rdds, directly from Google Storage Bucket.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs_for_dict = parquetFile.select("id", "title").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

Helper Functions in manipulating the rdds

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  tokens1 = [word for word in tokens if word not in all_stopwords]
  dic = Counter(tokens1)
  lst1 = []
  already = set()
  for word in tokens1:
    if word not in already:
      lst1.append((word,(id,dic.get(word))))
    already.add(word)
  return lst1

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  lst = sorted(unsorted_pl)
  return lst

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  rdd2 = postings.map(lambda a: (a[0], len(a[1])))
  return rdd2

def partition_postings_and_write(postings, folder_str):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE


  rdd2 = postings.groupBy(lambda a: token2bucket_id(a[0]))
  return rdd2.map(lambda a: InvertedIndex.write_a_posting_list(a,bucket_name, folder_str))


def sum_term_total(lst):
  sum = 0
  for i in lst:
    sum += i[1]
  return sum


def calculate_dl(word_counts):
  temp = word_counts.map(lambda x: (x[1][0],x[1][1]))
  dl_bydoc = temp.groupByKey()
  return dl_bydoc.mapValues(len)


def concat_anchor(idd, lst):
  string = ''
  for temp in lst:
    string = string + " " + temp
  x = Row(id=idd, text=string)
  return x

def nodup(idd, former):
  old = former.split()
  cur = ''
  for temp in old:
    if temp.lower() in cur:
      continue
    else:
      cur = cur + " " + temp.lower()
  x = Row(text=cur, id=idd)
  return x

calculations for body_inverted_index

In [ ]:
# word counts map
word_counts_body = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>50)
w2df_body = calculate_df(postings_filtered_body)
w2df_dict_body = w2df_body.collectAsMap()

dl_temp_body = calculate_dl(word_counts_body)

dl_map_body = dl_temp_body.collectAsMap()

termtotal = postings_filtered_body.map(lambda a: (a[0], sum_term_total(a[1])))

termtotal_map = termtotal.collectAsMap()

titles_map = doc_title_pairs_for_dict.collectAsMap()


# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered_body, 'body').collect()


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/.body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v)


In [ ]:
# Create inverted index instance
body_inverted_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
body_inverted_index.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
body_inverted_index.df = w2df_dict_body

body_inverted_index.DL = dl_map_body

body_inverted_index.term_total = termtotal_map

body_inverted_index.ids_titles_map = titles_map

# write the global stats out
body_inverted_index.write_index('.', 'body_inverted_index')
# upload to gs
index_src = "body_inverted_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/.body/{index_src}'
!gsutil cp $index_src $index_dst


In [ ]:
!gsutil ls -lh $index_dst

calculations for titles_inverted_index

In [ ]:
word_counts_titles = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

postings_titles = word_counts_titles.groupByKey().mapValues(reduce_word_counts)

w2df_titles = calculate_df(postings_titles)

w2df_dict_titles = w2df_titles.collectAsMap()

dl_temp_titles = calculate_dl(word_counts_titles)


dl_map_titles = dl_temp_titles.collectAsMap()


_ = partition_postings_and_write(postings_titles,'titles').collect()

In [ ]:
super_posting_locs_titles = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/.titles'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_titles[k].extend(v)

In [ ]:
# read page_rank

pr_path = f'gs://porjectirofererez111/pr_part-full.csv'
df = pd.read_csv(pr_path)

pr_dic = {}
pr_dic[int(df.columns[0])]=float(df.columns[1])
for row in df.iterrows():
    pr_dic[int(row[1][0])] = row[1][1]

In [ ]:
# Create inverted index instance
titles_inverted_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
titles_inverted_index.posting_locs = super_posting_locs_titles
# Add the token - df dictionary to the inverted index
titles_inverted_index.df = w2df_dict_titles

titles_inverted_index.DL = dl_map_titles

#uncomment to add pagerank
# titles_inverted_index.pr_dic = pr_dic

# write the global stats out
titles_inverted_index.write_index('.', 'titles_inverted_index')
# upload to gs
index_src = "titles_inverted_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/.titles/{index_src}'
!gsutil cp $index_src $index_dst


calculations for anchor_inverted_index

In [ ]:
just_anchors = parquetFile.select("anchor_text").rdd
just_anchors_flat = just_anchors.flatMap(lambda x: x).flatMap(lambda x: x)

just_anchors_flat_group = just_anchors_flat.groupByKey().mapValues(set)
just_anchors_flat_group_concat = just_anchors_flat_group.map(lambda x: concat_anchor(x[0],x[1]))

just_anchors_flat_group_concat_nodup = just_anchors_flat_group_concat.map(lambda x: nodup(x[0],x[1]))

word_counts_anchor = just_anchors_flat_group_concat_nodup.flatMap(lambda x: word_count(x[0], x[1]))

postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

w2df_anchor = calculate_df(postings_anchor)

w2df_dict_anchor = w2df_anchor.collectAsMap()

dl_temp_anchor = calculate_dl(word_counts_anchor)

dl_map_anchor = dl_temp_anchor.collectAsMap()


_ = partition_postings_and_write(postings_anchor, 'anchor').collect()

In [ ]:
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/.anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_anchor[k].extend(v)

In [ ]:

# Create inverted index instance
anchor_inverted_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
anchor_inverted_index.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
anchor_inverted_index.df = w2df_dict_anchor

anchor_inverted_index.DL = dl_map_anchor


# write the global stats out
anchor_inverted_index.write_index('.', 'anchor_inverted_index')
# upload to gs
index_src = "anchor_inverted_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/.anchor/{index_src}'
!gsutil cp $index_src $index_dst

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vertex (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # YOUR CODE HERE

  rdd4 = pages.flatMap(lambda a: a[1])
  rdd44 = rdd4.flatMap(lambda a: Row(a['id']))
  rdd444 = rdd44.map(lambda a: Row(a))

  rdd5 = pages.map(lambda a: Row(a[0]))

  vertices = rdd444.union(rdd5).distinct()

  edges = pages.flatMap(lambda a: [(a[0],p['id']) for p in a[1]]).distinct()

  return edges, vertices

Computes PageRank for the entire English Wikipedia.

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

In [ ]:
pages_links = parquetFile.select("id", "anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr.show()